In [ ]:
!pip install pyspark

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# Import necessary libraries
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:

from pyspark import SparkConf, SparkContext, SQLContext
import re

from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import classification_report
import numpy as np

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
import plotly.express as px

from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType, IntegerType
from textblob import TextBlob


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
conf = SparkConf()
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
train_rdd = sc.textFile("/content/drive/MyDrive/yelp_review_polarity_csv/train.csv")
test_rdd = sc.textFile("/content/drive/MyDrive/yelp_review_polarity_csv/test.csv")

# Convert to rdd
def split(line):
  label = int(line[1])
  sentence = line[5:-1]
  return label, sentence

train_rdd = train_rdd.map(split)
test_rdd = test_rdd.map(split)

# Get the first 10 rows
for row in train_rdd.take(10):
  print(row)      # label, text

(1, "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.")
(2, "Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. 

In [ ]:
# Preprocess
nltk.download('stopwords')

# Remove special characters
pattern = r'[^a-zA-Z0-9\s]'     # Define a pattern that only includes whitespaces and alphanumeric characters
def remove_special_characters(text):
    text = re.sub(pattern, ' ', text)     # Replace characters not belonging to the pattern with whitespace
    return text.replace('\n', ' ')      # Replace '\n' with whitespace

# Remove indepedent numbers and stopwords
def is_number(s):
    if s.isdigit():
        return True
    try:
        float(s)
        return True
    except:
        return False
stop_words = set(stopwords.words('english'))
stop_words.add('')
def remove_numbers_and_stopwords(text):
    return " ".join([x for x in text.split(' ') if not is_number(x) and x not in stop_words])

# Remove abundant spaces
def remove_extra_spaces(text):
    return " ".join(text.split(' '))

# Lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split(" ")])

def preprocess(text):
    text = remove_special_characters(text)
    text = text.lower()
    text = remove_numbers_and_stopwords(text)
    text = remove_extra_spaces(text)
    text = lemmatize(text)
    return text

# Remove rows with empty word lists
def filter_empty_and_none(row):
    return row[1] is not None and len(row[1]) > 0

train_preprocessed_rdd = (
    train_rdd
    .map(lambda x : (x[0], preprocess(x[1])))
    .filter(filter_empty_and_none)
)
test_preprocessed_rdd = (
    test_rdd
    .map(lambda x : (x[0], preprocess(x[1])))
    .filter(filter_empty_and_none)
)

for row in train_preprocessed_rdd.take(10):
  print(row)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(1, 'unfortunately frustration dr goldberg patient repeat experience many doctor nyc good doctor terrible staff seems staff simply never answer phone usually take hour repeated calling get answer time want deal run problem many doctor get office worker patient medical need anyone answering phone incomprehensible work aggravation regret feel give dr goldberg star')
(2, 'going dr goldberg year think one 1st patient started mhmg great year really big picture former gyn dr markoff found fibroid explores option patient understanding judge asks right question thorough want kept loop every aspect medical health life')
(1, 'know dr goldberg like moving arizona let tell stay away doctor office going dr johnson left goldberg took johnson left caring doctor interested co pay come medication refill every month give refill could le patient financial situation trying get day mail away pharmacy prescription guy joke make matter even worse office staff incompetent time call office put voice mail one e

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define schema for DataFrame
schema = StructType([
    StructField("label", IntegerType(), True),
    StructField("text", StringType(), True)
])

# Convert RDDs to DataFrames with specified schema
train_df = train_preprocessed_rdd.toDF(schema=schema)
test_df = test_preprocessed_rdd.toDF(schema=schema)

# Now you can use train_df and test_df with column names
train_df.show(10)

+-----+--------------------+
|label|                text|
+-----+--------------------+
|    1|unfortunately fru...|
|    2|going dr goldberg...|
|    1|know dr goldberg ...|
|    1|writing review gi...|
|    2|food great best t...|
|    1|wing sauce like w...|
|    1|owning driving ra...|
|    1|place absolute ga...|
|    2|finally made rang...|
|    2|drove yesterday g...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
# # Count the number of rows
# num_rows = train_df.count()

# print("Number of rows in reviews_df:", num_rows)

SVM - tfidf


In [ ]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from textblob import TextBlob

# Step 1: Prepare the Data
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures", numFeatures=12)
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="features")

# Additional Features: Sentiment Score
def analyze_sentiment(text):
    # Perform sentiment analysis using TextBlob
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    return sentiment_score

analyze_sentiment_udf = udf(analyze_sentiment, FloatType())
sentiment_data = train_df.withColumn("sentiment_score", analyze_sentiment_udf(train_df["text"]))

# Additional Feature: Word Length
word_length_udf = udf(lambda text: len(text.split()), IntegerType())
sentiment_data = sentiment_data.withColumn("word_length", word_length_udf(sentiment_data["text"]))

# Transformations
tokenized_data = tokenizer.transform(sentiment_data)
hashed_data = hashing_tf.transform(tokenized_data)
idf_data = idf.fit(hashed_data).transform(hashed_data)

# Selecting the columns to keep
combined_data = idf_data.select("label", "text", "sentiment_score", "word_length", "features")
combined_data.show(truncate=False)


+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.sql.functions import when
# Display unique values in the label column
combined_data.select("label").distinct().show()

# Define feature columns used for assembling features
feature_columns = ["sentiment_score", "word_length", "features"]

# Create a VectorAssembler to assemble features into a vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="assembled_features")

# Transform the combined_data DataFrame using the VectorAssembler
assembled_data = assembler.transform(combined_data)

# Adjust labels for binary classification (e.g., map label 2.0 to 1.0)
adjusted_data = assembled_data.withColumn("label_adjusted", when(assembled_data["label"] == 2.0, 1.0).otherwise(0.0))

# Select the assembled features and adjusted label
final_data = adjusted_data.select("assembled_features", "label_adjusted")

# Define Linear SVM classifier with SGD
svm = LinearSVC(labelCol="label_adjusted", featuresCol="assembled_features", maxIter=100, regParam=0.01)

# Train the Linear SVM classifier
model = svm.fit(final_data)

+-----+
|label|
+-----+
|    1|
|    2|
+-----+



In [ ]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.sql.functions import udf, length
from pyspark.sql.types import FloatType
from textblob import TextBlob

# Additional Features: Sentiment Score for Test Data
def analyze_sentiment(text):
    # Perform sentiment analysis using TextBlob
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    return sentiment_score

analyze_sentiment_udf = udf(analyze_sentiment, FloatType())
sentiment_test_data = test_df.withColumn("sentiment_score", analyze_sentiment_udf(test_df["text"]))

# Additional Features: Word Length for Test Data
word_length_udf = udf(lambda text: len(text.split()), IntegerType())
sentiment_test_data = sentiment_test_data.withColumn("word_length", word_length_udf(sentiment_test_data["text"]))

# Transformations for Test Data
tokenized_test_data = tokenizer.transform(sentiment_test_data)
hashed_test_data = hashing_tf.transform(tokenized_test_data)
idf_test_data = idf.fit(hashed_test_data).transform(hashed_test_data)

# Display the transformed test data
idf_test_data.show()
# Selecting the columns to keep
combined_data_test = idf_test_data.select("label", "text", "sentiment_score", "word_length", "features")
combined_data_test.show(truncate=True)

+-----+--------------------+---------------+-----------+--------------------+--------------------+--------------------+
|label|                text|sentiment_score|word_length|               words|         rawFeatures|            features|
+-----+--------------------+---------------+-----------+--------------------+--------------------+--------------------+
|    2|contrary review z...|     0.08611111|         54|[contrary, review...|(12,[0,1,2,3,4,5,...|(12,[0,1,2,3,4,5,...|
|    1|last summer appoi...|    0.016969698|         34|[last, summer, ap...|(12,[0,1,2,3,4,5,...|(12,[0,1,2,3,4,5,...|
|    2|friendly staff st...|     0.34166667|         11|[friendly, staff,...|(12,[0,2,5,6,7,8,...|(12,[0,2,5,6,7,8,...|
|    1|food good unfortu...|    0.033333335|         26|[food, good, unfo...|(12,[0,1,3,4,5,6,...|(12,[0,1,3,4,5,6,...|
|    2|even car filene b...|     0.18349282|         77|[even, car, filen...|(12,[0,1,2,3,4,5,...|(12,[0,1,2,3,4,5,...|
|    2|picture billy joe...|     0.30869

In [ ]:
from pyspark.sql.functions import when
from pyspark.ml.feature import VectorAssembler

# Define feature columns used for assembling features
feature_columns = ["sentiment_score", "word_length", "features"]

# Assemble features into a vector for test data
assembler = VectorAssembler(inputCols=feature_columns, outputCol="assembled_features")
assembled_test_data = assembler.transform(combined_data_test)

# Adjust labels for binary classification (e.g., map label 2.0 to 1.0)
adjusted_data_test = assembled_test_data.withColumn("label_adjusted", when(assembled_test_data["label"] == 2.0, 1.0).otherwise(0.0))

# Select the assembled features and adjusted label
final_test_data = adjusted_data_test.select("assembled_features", "label_adjusted")

# Train the SVM classifier on the test data
model = svm.fit(final_test_data)

# Optionally, you can use the trained model to make predictions on the test data
predictions = model.transform(final_test_data)

# Show predictions
predictions.show()


+--------------------+--------------+--------------------+----------+
|  assembled_features|label_adjusted|       rawPrediction|prediction|
+--------------------+--------------+--------------------+----------+
|[0.08611111342906...|           1.0|[0.70550486166818...|       0.0|
|[0.01696969754993...|           0.0|[1.05364175521960...|       0.0|
|[0.34166666865348...|           1.0|[-1.2855694998786...|       1.0|
|[0.03333333507180...|           0.0|[0.88094540779082...|       0.0|
|[0.18349282443523...|           1.0|[0.02491147850249...|       0.0|
|[0.30869317054748...|           1.0|[-1.2647261373403...|       1.0|
|[-0.1809523850679...|           0.0|[2.21572234474394...|       0.0|
|[0.12257539480924...|           0.0|[0.65769390549246...|       0.0|
|[0.02019230835139...|           0.0|[0.67507934614132...|       0.0|
|[0.24642856419086...|           1.0|[-0.6703771235143...|       1.0|
|[0.30256411433219...|           0.0|[-0.8633059756795...|       1.0|
|[0.36176469922065..

In [ ]:
# Step 6: Evaluate the Model
evaluator = MulticlassClassificationEvaluator(labelCol="label_adjusted", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = ", accuracy)

Accuracy =  0.7914627085636086


In [ ]:
# F1 score evaluation
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label_adjusted", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print("F1 Score = ", f1_score)

F1 Score =  0.7914348699710104


In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

# Convert predictions to RDD of (prediction, label) pairs
prediction_and_label = predictions.select("prediction", "label_adjusted").rdd.map(lambda row: (float(row.prediction), float(row.label_adjusted)))

# Instantiate MulticlassMetrics
metrics = MulticlassMetrics(prediction_and_label)

# Calculate precision, recall, and F1-score for each class
precision = {}
recall = {}
f1Score = {}

# Define the classes for which you want to calculate metrics
classes = [0.0, 1.0]  # Assuming binary classification with adjusted labels

for label in classes:
    precision[label] = metrics.precision(label)
    recall[label] = metrics.recall(label)
    f1Score[label] = metrics.fMeasure(label)

# Print the classification report
print("Precision by class:")
for label in classes:
    print("Class %s: %s" % (label, precision[label]))

print("Recall by class:")
for label in classes:
    print("Class %s: %s" % (label, recall[label]))

print("F1 Score by class:")
for label in classes:
    print("Class %s: %s" % (label, f1Score[label]))


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Precision by class:
Class 0.0: 0.7983297413793103
Class 1.0: 0.7849058545117811
Recall by class:
Class 0.0: 0.7799242025476366
Class 1.0: 0.803
F1 Score by class:
Class 0.0: 0.7890196496086054
Class 1.0: 0.7938498360996931


In [ ]:

# # Optionally, you can save the model for future use
model.save("/content/drive/MyDrive/big data models/svm_tfidf_addfeat_model1")


In [ ]:
from pyspark.ml.classification import LinearSVCModel

# Assuming 'svm_model' is your trained LinearSVCModel
assert isinstance(model, LinearSVCModel)

# Get feature coefficients (model weights)
coefficients = model.coefficients.toArray()

# Create a list of feature names (excluding 'label' column)
feature_columns = [col for col in combined_data.columns if col != 'label']

# Get total number of features
total_features = len(coefficients)

# Combine feature names with coefficients
feature_coefficient_list = list(zip(feature_columns, coefficients))

# Sort feature coefficient list by absolute coefficient value (descending order)
feature_coefficient_list_sorted = sorted(feature_coefficient_list, key=lambda x: -abs(x[1]))

# Print feature importance information
print("Total Number of Features:", total_features)
print("\nFeature Coefficients (Absolute Values):")
for feature, coefficient in feature_coefficient_list_sorted:
    print(f"{feature}: {abs(coefficient):.4f}")

# Highest Coefficient Feature
highest_coefficient_feature = feature_coefficient_list_sorted[0][0]
highest_coefficient_value = feature_coefficient_list_sorted[0][1]
print("\nFeature with Highest Coefficient:", highest_coefficient_feature)
print("Coefficient Value:", highest_coefficient_value)

# Lowest Coefficient Feature
lowest_coefficient_feature = feature_coefficient_list_sorted[-1][0]
lowest_coefficient_value = feature_coefficient_list_sorted[-1][1]
print("\nFeature with Lowest Coefficient:", lowest_coefficient_feature)
print("Coefficient Value:", lowest_coefficient_value)


Total Number of Features: 14

Feature Coefficients (Absolute Values):
text: 6.6390
features: 0.1982
word_length: 0.0091
sentiment_score: 0.0001

Feature with Highest Coefficient: text
Coefficient Value: 6.6390144996889076

Feature with Lowest Coefficient: sentiment_score
Coefficient Value: -6.88910056461836e-05


svm - word 2 vec


In [ ]:
# Step 1: Prepare the Data
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# Additional Features: Sentiment Score
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    return sentiment_score

analyze_sentiment_udf = udf(analyze_sentiment, FloatType())
sentiment_data = train_df.withColumn("sentiment_score", analyze_sentiment_udf(train_df["text"]))

# Additional Feature: Word Length
word_length_udf = udf(lambda text: len(text.split()), IntegerType())
sentiment_data = sentiment_data.withColumn("word_length", word_length_udf(sentiment_data["text"]))

# Tokenize data
tokenized_data = tokenizer.transform(sentiment_data)

# Step 2: Create Word2Vec Embeddings
word2vec = Word2Vec(vectorSize=200, seed=42, inputCol="words", outputCol="word2vec")
word2vec_model = word2vec.fit(tokenized_data)

# Transform the tokenized data to Word2Vec representation
word2vec_data = word2vec_model.transform(tokenized_data)

# Selecting the columns to keep
combined_data = word2vec_data.select("label", "text", "sentiment_score", "word_length", "word2vec")
combined_data.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import when

# Define Linear SVM classifier
svm = LinearSVC(labelCol="label_adjusted", featuresCol="assembled_features")

# Assemble features into a vector
feature_columns = ["sentiment_score", "word_length", "word2vec"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="assembled_features")
assembled_data = assembler.transform(combined_data)

# Adjust labels for binary classification (e.g., map label 2.0 to 1.0)
adjusted_data = assembled_data.withColumn("label_adjusted", when(assembled_data["label"] == 2.0, 1.0).otherwise(0.0))

# Select the assembled features and adjusted label
final_data = adjusted_data.select("assembled_features", "label_adjusted")

# Train the Linear SVM classifier
model = svm.fit(final_data)

In [ ]:
# Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# Sentiment Score Function
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    return sentiment_score

# UDF for Sentiment Analysis
analyze_sentiment_udf = udf(analyze_sentiment, FloatType())

# Apply Sentiment Analysis to Test Data
test_data_with_sentiment = test_df.withColumn("sentiment_score", analyze_sentiment_udf(test_df["text"]))

# Word Length UDF
word_length_udf = udf(lambda text: len(text.split()), IntegerType())

# Apply Word Length UDF to Test Data
test_data_with_word_length = test_data_with_sentiment.withColumn("word_length", word_length_udf(test_data_with_sentiment["text"]))

# Tokenize Test Data
tokenized_test_data = tokenizer.transform(test_data_with_word_length)

# Word2Vec Model
word2vec = Word2Vec(vectorSize=200, seed=42, inputCol="words", outputCol="word2vec")

# Fit Word2Vec Model on Training Data
word2vec_model = word2vec.fit(tokenized_data)

# Transform Test Data using Word2Vec Model
word2vec_test_data = word2vec_model.transform(tokenized_test_data)

# Select Columns to Keep
selected_columns_test = word2vec_test_data.select("label", "text", "sentiment_score", "word_length", "word2vec")

# Show the processed test data
selected_columns_test.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
feature_columns = ["sentiment_score", "word_length", "word2vec"]

# Assemble features into a vector for test data
assembler = VectorAssembler(inputCols=feature_columns, outputCol="assembled_features")
assembled_test_data = assembler.transform(selected_columns_test)

# Adjust labels for binary classification (e.g., map label 2.0 to 1.0)
adjusted_data_test = assembled_test_data.withColumn("label_adjusted", when(assembled_test_data["label"] == 2.0, 1.0).otherwise(0.0))

# Select the assembled features and adjusted label
final_test_data = adjusted_data_test.select("assembled_features", "label_adjusted")

# Train the SVM classifier on the test data
model = svm.fit(final_test_data)

# Optionally, you can use the trained model to make predictions on the test data
predictions = model.transform(final_test_data)

# Show predictions
predictions.show()

+--------------------+--------------+--------------------+----------+
|  assembled_features|label_adjusted|       rawPrediction|prediction|
+--------------------+--------------+--------------------+----------+
|[0.08611111342906...|           1.0|[2.04437626008547...|       0.0|
|[0.01696969754993...|           0.0|[2.55726300523968...|       0.0|
|[0.34166666865348...|           1.0|[-1.2636982171925...|       1.0|
|[0.03333333507180...|           0.0|[2.75413278835891...|       0.0|
|[0.18349282443523...|           1.0|[-0.8116447365465...|       1.0|
|[0.30869317054748...|           1.0|[-2.8117731980063...|       1.0|
|[-0.1809523850679...|           0.0|[9.13540381648191...|       0.0|
|[0.12257539480924...|           0.0|[1.47501535082082...|       0.0|
|[0.02019230835139...|           0.0|[-0.7392361388124...|       1.0|
|[0.24642856419086...|           1.0|[-4.4487586846823...|       1.0|
|[0.30256411433219...|           0.0|[1.67545091779813...|       0.0|
|[0.36176469922065..

In [ ]:
# Step 6: Evaluate the Model
evaluator = MulticlassClassificationEvaluator(labelCol="label_adjusted", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = ", accuracy)

Accuracy =  0.9154955523974946


In [ ]:
# F1 score evaluation
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label_adjusted", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print("F1 Score = ", f1_score)

F1 Score =  0.9154955187443157


In [ ]:

# # Optionally, you can save the model for future use
model.save("/content/drive/MyDrive/big data models/svm_word2vec_addfeat_model1")